# Filtering and Downloading Scenes as a GeoPackage

In this tutorial, we will use the catalog download resource to get the GeoPackage schema, then to discover pages of published scenes from which we will populate an in-memory GeoPackage collection using [fiona](https://github.com/Toblerity/Fiona), and finally we will download the populated GeoPackage. See the [GeoPackage download API documentation](https://api.carbonmapper.org/api/v1/docs#/Data%20Catalog/catalog_api_scene_scenes_download_gpkg).

The resource provides the `intersects` parameter, which accepts a feature's GeoJSON geometry and will filter for scenes that intersect the provided geometry. Our GeoJSON is a simplified representation of the state of California. Note that there is an 8,192 byte API URL size limit for requests, hence the simplified GeoJSON. We will also provide a 3-year date range for our query. Because the endpoint is paginated, the `count` returned may differ from the length of the `items` returned.

## Installing Requirements

For tutorial purposes, we will use IPython's [pip](https://ipython.readthedocs.io/en/stable/interactive/magics.html#magic-pip) magic command to install the [Fiona](https://github.com/Toblerity/Fiona) and [Requests](https://pypi.org/project/requests/) requirements in the current kernel.

In [1]:
%pip install fiona requests

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 36.0 MB/s eta 0:00:00m eta 0:00:010:01:01
Note: you may need to restart the kernel to use updated packages.


## Requesting an Access Token

The catalog download endpoint requires a platform API token. The [STAC token tutorial](https://github.com/carbon-mapper/platform-public/blob/main/tutorials/scoped_token.ipynb) provides a process to create, store, and print a scoped token. The STAC token tutorial can be used to store the token with IPython. If the token exists in IPython storage, we ask to use it. If it hasn't been stored, or the user declines to use it, we ask the user to provide a token.

Information about manually creating a scoped token can be found in the tutorials [README](https://github.com/carbon-mapper/platform-public/tree/main/tutorials#api-authentication).

In [2]:
# Retrieve IPython storage
%store -r

# Try to get a scoped token
try:
    # The token can be retrieved from IPython magic storage if the user elected to store it in the STAC token tutorial
    token = cm_scoped_token["token_value"]
except NameError:
    # The token does not exist in magic storage
    token = None
else:
    # The token exists in magic storage
    if input("Existing scoped token found in IPython storage. Would you like to use it? (Y/n)? ").lower() != "y":
        token = None
finally:
    if not token:
        # Allow the token value to be provided by the user if it was not found in magic storage
        token = input("Enter your Carbon Mapper platform API scoped token: ")

if not token:
    raise ValueError("A scoped token is required for this tutorial.")

## Filtering and Downloading Scenes

In [3]:
import os
import pathlib

import requests
from fiona import Feature
from fiona.io import MemoryFile

geopackage_path = "catalog/download/scenes.gpkg"
base_url = "https://api.carbonmapper.org/api/v1/"
payload = {
    "intersects": '{"type": "MultiPolygon", "coordinates": [[[[-124.346415, 42.041457], [-125.981193, 41.962186], [-124.607620, 31.606420], [-114.338004, 32.468868], [-114.068746, 41.913432], [-124.346415, 42.041457]]]]}',
    "datetime": "2020-01-01/2023-12-31",
    "offset": 0,
}
# Full path at which to store the file
# If empty, will save to current working directory
file_path = pathlib.Path("")
file_name = "scenes.gpkg"
abs_path = os.path.abspath(file_path / file_name)
headers = {"Authorization": f"Bearer {token}"}
print("Downloading features.")

# Request a GeoPackage with 0 features
response = requests.get(
    f"{base_url}{geopackage_path}",
    params={"limit": 0},
    headers=headers,
)
response.raise_for_status()

# Get the schema from the empty GeoPackage
# The filename argument is not required, but is included to suppress a confusing warning
with MemoryFile(response.content, filename="schema_file.gpkg") as schema_file:
    with schema_file.open() as schema_collection:
        schema = schema_collection.schema

with MemoryFile() as dst_file:
    # Instantiate an  in-memory GeoPackage using the previously retrieved schema
    with dst_file.open(mode="w", driver="GPKG", schema=schema) as dst_collection:
        while True:
            # Download a page of features
            response = requests.get(
                f"{base_url}{geopackage_path}",
                params=payload,
                headers=headers,
            )
            response.raise_for_status()

            count = int(response.headers["pagination-count"])
            limit = int(response.headers["pagination-limit"])
            offset = int(response.headers["pagination-offset"])

            # Load the source GeoPackage into an in-memory file
            with MemoryFile(response.content) as src_file:
                with src_file.open() as src_collection:
                    # Add the source features into the destination collection
                    for feature in src_collection:
                        dst_collection.write(Feature(geometry=feature.geometry, properties=feature.properties))

                    print(f"Added {len(list(src_collection))} features to scenes GeoPackage.") 


            payload["offset"] = int(offset + limit)

            # No more records to download
            if payload["offset"] >= count:
                break

    confirm = input("{action} file ({abs_path})? Y/n".format(
        action="Overwrite" if pathlib.Path(abs_path).is_file() else "Save",
        abs_path=abs_path,
    ))

    if confirm.lower() == "y":
        # Save the GeoPackage to disk
        with open(abs_path, "wb") as file:
            file.write(dst_file.getbuffer())

        print(f"Scenes downloaded to {abs_path}.")


Added 1000 features to scenes GeoPackage.
Added 1000 features to scenes GeoPackage.
Added 383 features to scenes GeoPackage.
